<a href="https://colab.research.google.com/github/anmutechsupport/AD-biomarker-optimization/blob/main/PreClinicalModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import re

In [ ]:
testList = ['Brief Title', 'Brief Summary', 'Detailed Description', 'Study Type', 
           'Study Design', 'Condition', 'Intervention', 'Eligibility Criteria']

# Get Endings to Links of other cancer papers

In [ ]:
link_df = pd.read_csv("full_ids.csv")

In [ ]:
URLS = []
full_data = []

# Creates the links to each paper
for i in range(len(link_df)):
  id = link_df.loc[i, 'NCTId']
  URL = "https://clinicaltrials.gov/ct2/show/record/" + id
  URLS.append(URL)

# datalinkdf = pd.DataFrame(URLS)

`df.join()` only works for dataframes, but since we're storing the columns into a list it won't work. (We'd have to convert the list back into a dataframe and then join them which might take too long)

In [ ]:
# Iterate through each link and finds the table, gets necessary info and adds it
# to the full_data list
all_data = []
for j in range(len(link_df)):

    id = link_df.loc[j, 'NCTId']
    link = URLS[j]
    dfs = pd.read_html(link)
    df = dfs[2]
    # Leaves requested features only
    # Renames second column to its id
    df = df.rename(index=str, columns={"Tracking Information.1": id})
    # gets rid of the index numbers 
    df.set_index("Tracking Information", inplace=True)
    # Reverses columns and rows
    df = df.transpose()
    print(df)

    # add each paper data into one list
    full_data.append(df)


    # looping through each name in the test list
    savedValues = []
    try:
      dropped = [df.pop(i) for i in testList]
        # concatted.columns = testList
    except:
      print('skipped')
      pass

    concatted = pd.DataFrame(pd.concat(dropped))
    concatted = concatted.transpose()

    try:
      concatted.columns = testList
    except:
      print(concatted)
      print(concatted.columns)
      print(len(concatted.columns))
      print(concatted.shape)
      raise Exception
    all_data.append(concatted)
  
allDataDf = pd.concat(all_data)
print("Completed")
uniqueDf = allDataDf.drop_duplicates().to_csv('allUniqueData.csv')

Streaming output truncated to the last 5000 lines.
Tracking Information First Submitted Date ICMJE  ... ICMJE Data element required by the  International Committee of Medical Journal Editors and the  World Health Organization ICTRP
NCT02314286                   November 16, 2014  ...  ICMJE Data element required by the  Internatio...                                                                             

[1 rows x 54 columns]
skipped
Tracking Information First Submitted Date  ... Verification Date
NCT04313998                March 11, 2020  ...        March 2020

[1 rows x 56 columns]
Tracking Information First Submitted Date  ... Verification Date
NCT03073434             February 13, 2017  ...        April 2020

[1 rows x 56 columns]
Tracking Information First Submitted Date  ... Verification Date
NCT03567551                  May 21, 2018  ...     February 2021

[1 rows x 56 columns]
Tracking Information First Submitted Date ICMJE  ... ICMJE Data element required by the  Internat

HTTPError: ignored

In [ ]:
allDataDf = pd.concat(all_data)

In [ ]:
# fetch the data and get the descriptions
allData = pd.read_csv('allUniqueData (1).csv')
descriptions = allData['Detailed Description']

In [ ]:
class Embed(nn.Module):
  def __init__(self, vocab):
    super(Embed, self).__init__()
    self.emb = nn.Embedding(len(vocab.keys()), 100)
  
  def forward(self, x):

    return self.emb(x)

In [ ]:
def embedTextualFeature(descriptions):

  embeddedFeatures = []
  parsedDescriptions = []
  for desc in descriptions:
    # remove punctuation
    stripped = re.sub(r'[^\w\s]', '', desc)
    # make all lowercase
    parsedDescriptions.append(stripped.lower())
  # used parsed descriptions to build vocabulary
  vocab = {word:ixWord for ixWord, word in enumerate(sorted(set(" ".join(parsedDescriptions).split(" "))))}
  # tokenize the words
  tokenizedDescriptions = []
  for desc in parsedDescriptions:
    tokenizedDescriptions.append([vocab[word] for word in desc.split(" ")])
  
  emb = Embed(vocab)
  for desc in tokenizedDescriptions:
    out = emb(torch.tensor(desc).long())
    embeddedFeatures.append(out)
  
  return embeddedFeatures

In [ ]:
emb = embedTextualFeature(descriptions)

In [ ]:
class BiomarkerClassifier(nn.Module):

  def __init__(self, in_size, hidden, nPredictive, nSurrogate):
    super(BiomarkerClassifier, self).__init__()
    self.fc1 = nn.Linear(in_size, hidden)
    self.predictiveEndpoint  = nn.Linear(hidden, nPredictive)
    self.surrogateEndpoint = nn.Linear(hidden, nSurrogate)

  def forward(self, x):
    # fix this to work with all features
    x = self.fc1(x)
    pred = self.predictiveEndpoint(x)
    sur = self.surrogateEndpoint(x)

    return pred, sur


In [ ]:

model = BiomarkerClassifier(in_size = 100, hidden = 50)
optim = optim.Adam(model.parameters())
criteron = nn.MSELoss()

def train(modelInstance, embeddedFeatures):

  for feature, target in zip(embeddedFeatures, results):

    optim.zero_grad()
    pred, sur = modelInstance(feature)
    predTarget, surTarget = target[0], target[1]

    predLoss = criterion(predTarget, pred)
    surLoss = criteron(surTarget, sur)

    totalLoss = predLoss + surLoss 
    totalLoss.backward()

    optim.step()